Final Project Notebook

In [ ]:
# SETUP CODE
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 10)

<h2>Introduction</h2>

Bank managers have been asking the same question for a long time. What sort of clients will subscribe to a term deposit? To solve this question, we have downloaded the “bank-additional-full.csv” spreadsheet from the Portuguese banking institution to anlysis the results and conduct a research. By using the KNN classification method, we aim to predict if the client will subscribe to a term deposit or not. 

"Bank-additional-full.csv” is a document representing the marketing efforts of the Portuguese banking institution. 21 variables are measured, with the classification variable being whether or not a client will subscribe to a term deposit. A term deposit is a fixed-term investment that contains the deposit of funds into an account at a financial institution, and receiving a certain amount of interest at maturity date. This data was originally produced by researchers at the University Institute of Lisbon for a report highlighting a data-driven approach to predict the successfulness of bank telemarketing.

This dataset features 20 attributes regarding demographics (age, job, marital status), economic and educational situation (level of education, possession of a housing loan), and historical information’s relationship to the banking institution (last month of contact, outcome from previous marketing campaign). In order to make this prediction more effective and accurate, we have selected the 7 most important predictors to conduct KNN classification analysis, and each of the target variables contain 41188 observations, which makes the whole prediction more trustworthy and accurate. We will separate 75% of the data into training set and 25% of the remaining into the testing set in order to find out the most suitable k-neighbour in our function. After identifiying the correct "k" for the function, we will use "ggpairs" to create a visual comparasion to demonstrates the correlation  of each variables measured, 


In [ ]:
# Retrieving and preparing the dataset for classification

# Set the seed
set.seed(7134) 

# Read the dataset and discard the unnecessary columns
temp <- tempfile()
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip",temp)
bank_data <- read.table(unz(temp, "bank-additional/bank-additional-full.csv"), sep=";", header=TRUE) %>%
    select(age, job, marital, education, housing, loan, campaign, y)
unlink(temp)
bank_data <- as_tibble(bank_data)
bank_data
nrow(bank_data)

In [ ]:
# Set the seed
set.seed(7134) 

# Filter out rows with unknown data
bank_data_filtered <- bank_data %>%
    filter(job != "unknown",
           marital != "unknown",
           education != "unknown",
           housing != "unknown",
           loan != "unknown")
             
# Convert all string columns to integers (needed for classification)
bank_data_nums <- bank_data_filtered %>%
    mutate(
        age = as.double(age),
        job = as.double(unclass(factor(job))), 
        marital = as.double(unclass(factor(marital))), 
        education = as.double(unclass(factor(education))), 
        housing = as.double(unclass(factor(housing))),
        loan = as.double(unclass(factor(loan))),
        y = factor(y)
    )
bank_data_nums

In [ ]:
# Set the seed
set.seed(7134) 

# Split the dataset into training and testing sets
bank_split <- initial_split(bank_data_nums, prop = 0.75, strata = y)  
bank_train <- training(bank_split)   
bank_test <- testing(bank_split)
bank_test
bank_train

In [ ]:
# Perform cross validation to determine the ideal # of neighbours

# Set the seed
set.seed(7134) 

# No need to use the entire dataset for this, 
# especially since cross validation would take too long on a datset so large
bank_sample <- rep_sample_n(bank_train, 1000)

bank_k_recipe <- recipe(y ~ age+job+education+housing+loan+marital+campaign, data = bank_sample) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

bank_vfold <- vfold_cv(bank_sample, v = 5, strata = y)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
       set_engine("kknn") %>%
       set_mode("classification")
knn_spec

In [ ]:
# Set the seed
set.seed(7134) 
bank_knn_results <- workflow() %>%
       add_recipe(bank_k_recipe) %>%
       add_model(knn_spec) %>%
       tune_grid(resamples = bank_vfold, grid = 20) %>%
       collect_metrics()
bank_knn_results

In [ ]:
# Plot the accuracies of each k determined by cross validation
# to find the ideal value of k
options(repr.plot.width=10, repr.plot.height=10)

accuracies <- bank_knn_results %>% 
       filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate") +
       ggtitle("Accuracy Estimates for Number of Neighbours") +
       theme(text = element_text(size = 20)) +
       scale_x_continuous(breaks = seq(0, 14, by = 1)) +  # adjusting the x-axis
       scale_y_continuous(limits = c(0.4, 1.0)) # adjusting the y-axis

accuracy_versus_k

In [ ]:
# Comments from the TA
# How can we make sure the data does not include any missing values? Make sure to include a code checking for missing data. 

From the above plot, we can see that although having 9-14 neighbours would technically be the most accurate, the accuracy of any k greater than 2 is practically the same, therefore we will choose k = 3, since any k lower would be noticeably less accurate, and any k greater would be just as accurate, but require more computation time.

In [ ]:
bank_recipe <- recipe(y ~ age+job+education+housing+loan+marital+campaign, data = bank_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())
bank_recipe

bank_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) %>%
       set_engine("kknn") %>%
       set_mode("classification")

bank_spec

bank_fit <- workflow() %>%
        add_recipe(bank_recipe) %>%
        add_model(bank_spec) %>%
        fit(data = bank_train)
bank_fit

In [ ]:
bank_predictions <- predict(bank_fit , bank_test) %>%
       bind_cols(bank_test)
bank_predictions

bank_metrics <- bank_predictions %>%
         metrics(truth = y, estimate = .pred_class)
bank_metrics

bank_conf_mat <- bank_predictions %>% 
       conf_mat(truth = y, estimate = .pred_class)
bank_conf_mat

In [ ]:
library(GGally)
plot_pairs <- bank_data_nums %>%
  select(age, job, marital, education, housing, loan, campaign, y,)%>%
  ggpairs
plot_pairs